In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from matplotlib import rcParams
import requests
import time
import os

В этом блокноте мы скачиваем биржевую стоимость акций различных сырьевых компаний и некоторые фьючерсы при помощи yahoo finance api.
Из execel файла matrials скаченного с yahoo мы берем наименования сырьевых компаний.

Мы сохраняем биржевые показатели в папку basic_matrials_load, а фьючерсы в basic_futures_and_other_load.



Используя бесплатный апи ключ yahoo, мы ограничены 500 запросами.

In [2]:
def parce_the_data(host, key, interval, name, range_of_data, region):
    url = "https://yh-finance.p.rapidapi.com/stock/v2/get-chart"

    querystring = {"interval":interval,"symbol":name,"range":range_of_data,"region":region}

    headers = {
        'x-rapidapi-host': host,
        'x-rapidapi-key': key
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    return response.json()
  
def parseTimestamp(inputdata):
    timestamplist = []

    timestamplist.extend(inputdata["chart"]["result"][0]["timestamp"])
    timestamplist.extend(inputdata["chart"]["result"][0]["timestamp"])

    calendertime = []

    for ts in timestamplist:
        dt = datetime.fromtimestamp(ts)
        calendertime.append(dt.strftime("%m/%d/%Y"))
        
    return calendertime

def parseValues(inputdata):
    valueList = []
    valueList.extend(inputdata["chart"]["result"][0]["indicators"]["quote"][0]["open"])
    valueList.extend(inputdata["chart"]["result"][0]["indicators"]["quote"][0]["close"])

    return valueList

def attachEvents(inputdata):
    eventlist = []
    for i in range(0,len(inputdata["chart"]["result"][0]["timestamp"])):
        eventlist.append("open")  

    for i in range(0,len(inputdata["chart"]["result"][0]["timestamp"])):
        eventlist.append("close")

    return eventlist

def make_data_frame_with_history(host, key, interval, name, range_of_data, region):
    response = parce_the_data(host, key, interval, name, range_of_data, region)
    inputdata = pd.DataFrame()
    inputdata["Timestamp"] = parseTimestamp(response)
    inputdata["Values"] = parseValues(response)
    inputdata["Events"] = attachEvents(response)

    df= pd.DataFrame(inputdata)
    df = df[df['Events']=='open']
    df = df.drop('Events',axis=1)
    df = df.rename(columns = {'Values':name} )

    
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df = df.drop_duplicates()
    df = df.drop_duplicates(subset = 'Timestamp')
    idx = pd.date_range(str(df['Timestamp'].min()), str(df['Timestamp'].max()))
    df = df.set_index('Timestamp')
    df.index = pd.DatetimeIndex(df.index)
    df = df.reindex(idx)
    df=df.fillna(method='ffill')
    
    return df
            

In [3]:
#используя эту функцию мы получаем датафрейм с ежедневной ценой выбранной позиции на момент открытия торгов
#каждый день за последние 10 лет


host = "yh-finance.p.rapidapi.com"
key = "e7d7e7bd25msha8134841232e012p1c4bd4jsnc23ef6c6cd42"

df = make_data_frame_with_history(host, key, "1d", 'IRON.TO', "10y", "US")
df

,IRON.TO
2012-03-19,3.150
2012-03-20,3.060
2012-03-21,2.970
2012-03-22,2.860
2012-03-23,2.940
...,...
2022-03-13,0.075
2022-03-14,0.075
2022-03-15,0.075
2022-03-16,0.075


In [40]:
names_of_companies= list(pd.read_excel('basic_materials.xlsx')['Symbol'].unique())

for name in names_of_companies:
    if name+'.csv' not in os.listdir('basic_matrials_load/'):
        print(name)
        data = make_data_frame_with_history(host, key, "1d", name, "10y", "US")
        data.to_csv('basic_matrials_load/'+name+'.csv')
        time.sleep(5)

In [41]:
data = make_data_frame_with_history(host, key, "1d", 'RUB=X', "10y", "US")
data.to_csv('RUB=X.csv')

In [45]:
names_of_futures = ['GC=F','HDG=F','HG=F','CL=F','HO=F', '^STEEL', 'SI=F','RUB=X']

for name in names_of_futures:
    if name+'.csv' not in os.listdir('basic_futures_and_other_load/'):
        print(name)
        data = make_data_frame_with_history(host, key, "1d", name, "10y", "US")
        data.to_csv('basic_futures_and_other_load/'+name+'.csv')
        time.sleep(5)

RUB=X
